<a href="https://colab.research.google.com/github/ziqi9307/generative_chatbots/blob/main/Generative_Chatbots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Generative Chatbots Using Reccurent Neural Nets

### Download Required Files

In [16]:
!curl -O https://content.codecademy.com/programs/chatbots/generative-chatbots/twitter-project.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11552  100 11552    0     0  62782      0 --:--:-- --:--:-- --:--:-- 62782


In [2]:
!ls

sample_data  twitter-project.zip


In [17]:
!unzip -qq twitter-project.zip

In [19]:
!rm -f twitter-project.zip

In [20]:
!ls

__MACOSX  twitter-project


In [21]:
!rm -r __MACOSX/

In [14]:
!rm -r sample_data

In [24]:
!mv twitter-project/* .

In [25]:
!rm -r twitter-project/

### Preprocessing

**Preprocessing**<br>
Open twitter_prep.py and preprocessing.py in a code editor or IDE.
In twitter_prep.py, change data_path to the file path of [your-topic].txt. If it’s in the same directory as twitter_prep.py, then all you need is the file name. <br>

At the bottom of twitter_prep.py, print out all the response pairs from the data. Run the file to see the format they are in. Then comment the print statement out. <br>

In preprocessing.py, adjust the number of lines of training data you want to work with. We’re giving you a default of 15, but depending on how much you want to tax your computer, you can go up to -1 (the last line of the file).
Run preprocessing.py and make sure everything works, error-free.

In [ ]:
from itertools import zip_longest
import re


data_path = "twitter-project/cat.txt"

# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')


lines = [re.sub(r"(?:\@|https?\://)\S+", "", line).strip() for line in lines]

# group lines by response pair
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

pairs = list(grouper(lines, 2))
len(pairs)

26

twitter-project/cat.txt


### Training

Open training_model.py.
Change the values for the following: <br><br>
**batch_size:** You can choose to adjust this or not at this point. This determines how many response pairs are used at a time for training. <br>
**epochs:** This should be a larger number so that the seq2seq model has many chances to improve. Bear in mind that a larger number of epochs will also take your computer a lot longer to process. If you don’t have the ability to leave your computer running awhile for this project, then choose a number that is less than 500. <br><br>
<hr><br>

Run the code to generate your model. In the terminal, you should see a summary of the model printed out. Meanwhile, you’ll also see a new file in the directory called training_model.h5. This is where your seq2seq training model is saved so that it’s quicker for you to run your code when you instantiate the chatbot. <br>
Note that you may get the following error when attempting to run your program on a regular computer that uses CPU processing:

OMP: Error #15: Initializing libiomp5.dylib, but found libiomp5.dylib already initialized.
OMP: Hint: This means that multiple copies of the OpenMP runtime have been linked into the program. That is dangerous, since it can degrade performance or cause incorrect results. The best thing to do is to ensure that only a single OpenMP runtime is linked into the process, e.g. by avoiding static linking of the OpenMP runtime in any library. As an unsafe, unsupported, undocumented workaround you can set the environment variable KMP_DUPLICATE_LIB_OK=TRUE to allow the program to continue to execute, but that may cause crashes or silently produce incorrect results. For more information, please see http://www.intel.com/software/products/support/.
Abort trap: 6

In [ ]:
from preprocessing import num_encoder_tokens, num_decoder_tokens, decoder_target_data, encoder_input_data, decoder_input_data, decoder_target_data, max_encoder_seq_length, max_decoder_seq_length

from tensorflow import keras
# Add Dense to the imported layers
from keras.layers import Input, LSTM, Dense, Masking
from keras.models import Model
import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Choose dimensionality
dimensionality = 256

# Choose the batch size
# and number of epochs:
batch_size = 10
epochs = 50

# Encoder training setup
encoder_inputs = Input(shape=(None, num_encoder_tokens))

encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]

# Decoder training setup:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Building the training model:
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model:
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')

# print("Training the model:\n")
# Train the model:
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)

training_model.save('training_model.h5')